# Importing libraries

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from tqdm import tqdm

# Model definition

In [6]:
class VariationalAutoEncoder(nn.Module):
  def __init__(self, input_dim, h_dim, z_dim):
    super().__init__()
    # encoder
    self.img_2hid = nn.Linear(input_dim, h_dim)
    self.hid_2mu = nn.Linear(h_dim, z_dim)
    self.hid_2sigma = nn.Linear(h_dim, z_dim)

    # decoder
    self.z_2hid = nn.Linear(z_dim, h_dim)
    self.hid_2img = nn.Linear(h_dim, input_dim)

    self.relu = nn.ReLU()                                 # activation func in hidden layers

  def encode(self, x):
    h = self.relu(self.img_2hid(x))
    mu, sigma = self.hid_2mu(h), self.hid_2sigma(h)
    return mu, sigma

  def decode(self, z):
    h = self.relu(self.z_2hid(z))
    return self.hid_2img(h)

  def forward(self, x):
    mu, sigma = self.encode(x)
    epsilon = torch.randn_like(sigma)
    z_new = mu + sigma*epsilon                            # reparameterization trick
    x_reconstructed = self.decode(z_new)
    return x_reconstructed, mu, sigma

# Luca experiments (representing data by hour)

### Energy surplus / deficit by Noemi file

In [68]:
filename = f"lcpb_files/sbil.zst"
class_data = pd.read_parquet(filename)

# sel NORD
class_data = class_data[class_data['macrozone'] == 'NORD']
# take timestamp and set as index
class_data['timestamp'] = (class_data['date_time'])
class_data = class_data.set_index('timestamp')
# resample every 3h
resampled = class_data.resample('3h').agg({'sign': 'sum'}) # lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan})
resampled = resampled.reset_index()

# remove 2025
resampled = resampled[resampled['timestamp'].dt.year != 2025]

resampled

,timestamp,sign
0,2022-01-01 00:00:00+00:00,-12
1,2022-01-01 03:00:00+00:00,-12
2,2022-01-01 06:00:00+00:00,-12
3,2022-01-01 09:00:00+00:00,-12
4,2022-01-01 12:00:00+00:00,-12
...,...,...
8763,2024-12-31 09:00:00+00:00,12
8764,2024-12-31 12:00:00+00:00,10
8765,2024-12-31 15:00:00+00:00,-2
8766,2024-12-31 18:00:00+00:00,8


### Data import

In [184]:
df = pd.read_csv("output/IT_NORD_VAE_input.csv")
df = df.drop(columns = "Unnamed: 0")
df

,year_t0,2t_t0,solar_t0,tp_t0,ws_10m_t0,ws_100m_t0,hour_sin_t0,hour_cos_t0,day_sin_t0,day_cos_t0,...,ws_100m_t8,hour_sin_t8,hour_cos_t8,day_sin_t8,day_cos_t8,month_sin_t8,month_cos_t8,sin_dayofyear_t8,cos_dayofyear_t8,timestamp
0,-1.224855,-1.266622,-0.785197,-0.540314,-0.169929,0.237315,0.249009,1.890047,0.280164,1.400589,...,-1.132097,-0.685682,1.408711,0.280448,1.400505,0.702453,1.109234,0.025586,1.291842,2022-01-01 00:00:00
1,-1.224855,-1.324489,-0.785197,-0.540314,-0.167036,0.001132,1.183829,1.408519,0.280164,1.400589,...,-0.931334,0.249115,1.890286,0.552704,1.314595,0.702453,1.109234,0.050415,1.291222,2022-01-01 03:00:00
2,-1.224855,-1.363120,-0.691746,-0.543042,-0.218606,-0.093139,1.571044,0.246007,0.280164,1.400589,...,-0.778698,1.183912,1.408711,0.552704,1.314595,0.702453,1.109234,0.050415,1.291222,2022-01-01 06:00:00
3,-1.224855,-1.091436,0.355950,-0.543042,-0.396948,-0.530006,1.183829,-0.916506,0.280164,1.400589,...,-0.664870,1.571118,0.246088,0.552704,1.314595,0.702453,1.109234,0.050415,1.291222,2022-01-01 09:00:00
4,-1.224855,-0.496955,0.545301,-0.543042,-0.719834,-0.962998,0.249009,-1.398034,0.280164,1.400589,...,-0.452988,1.183912,-0.916535,0.552704,1.314595,0.702453,1.109234,0.050415,1.291222,2022-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8315,1.224561,-0.453550,-0.478673,-0.546437,-1.220757,-1.408609,-0.685811,-0.916506,-0.287572,1.400589,...,-1.518295,0.249115,-1.398110,-0.003431,1.429539,-0.019310,1.295919,0.000749,1.292048,2024-12-30 15:00:00
8316,1.224561,-1.110274,-0.785197,-0.545773,-1.125086,-1.549938,-1.073026,0.246007,-0.287572,1.400589,...,-1.388240,-0.685682,-0.916535,-0.003431,1.429539,-0.019310,1.295919,0.000749,1.292048,2024-12-30 18:00:00
8317,1.224561,-1.110274,-0.785197,-0.545773,-1.125086,-1.549938,-1.073026,0.246007,-0.287572,1.400589,...,-1.439442,-1.072887,0.246088,-0.003431,1.429539,-0.019310,1.295919,0.000749,1.292048,2024-12-30 18:00:00
8318,1.224561,-1.322641,-0.785197,-0.545773,-1.180936,-1.527834,-0.685811,1.408519,-0.287572,1.400589,...,-1.439442,-1.072887,0.246088,-0.003431,1.429539,-0.019310,1.295919,0.000749,1.292048,2024-12-30 21:00:00


In [185]:
# keeping only dayofthe year
features = ['day_sin_t0','day_cos_t0',
           'day_sin_t1','day_cos_t1',
           'day_sin_t2','day_cos_t2',
           'day_sin_t3','day_cos_t3',
           'day_sin_t4','day_cos_t4',
           'day_sin_t5','day_cos_t5',
           'day_sin_t6','day_cos_t6',
           'day_sin_t7','day_cos_t7',
           'day_sin_t8','day_cos_t8',
           'month_sin_t0','month_cos_t0',
           'month_sin_t1','month_cos_t1',
           'month_sin_t2','month_cos_t2',
           'month_sin_t3','month_cos_t3',
           'month_sin_t4','month_cos_t4',
           'month_sin_t5','month_cos_t5',
           'month_sin_t6','month_cos_t6',
           'month_sin_t7','month_cos_t7',
           'month_sin_t8','month_cos_t8']

# in alternative one can use the following lines to keep day and month
'''
features = ['sin_dayofyear_t0','cos_dayofyear_t0',
            'sin_dayofyear_t1','cos_dayofyear_t1',
            'sin_dayofyear_t2','cos_dayofyear_t2',
            'sin_dayofyear_t3','cos_dayofyear_t3',
            'sin_dayofyear_t4','cos_dayofyear_t4',
            'sin_dayofyear_t5','cos_dayofyear_t5',
            'sin_dayofyear_t6','cos_dayofyear_t6',
            'sin_dayofyear_t7','cos_dayofyear_t7',
            'sin_dayofyear_t8','cos_dayofyear_t8'
            ]
'''

df = df.drop(columns = features)
timestamp = pd.to_datetime(df['timestamp'])

timestamp_backup = timestamp.copy()

################################################
# SOFIA
# questo vettore (timestamp) è quello che puoi usare per colorare diversamente i punti secondo un certo criterio
# al momento sta usando i dati di produzione di noemi (quelli un po' scrausi)
# se vuoi vedere cosa succede per le diverse stagioni puoi provare a cambiare con queste righe:
#timestamp = timestamp.dt.hour
#timestamp = [1 if x in [1,2,3] else 2 if x in [4,5,6] else 3 if x in [7,8,9] else 4 if x in [10,11,12] for x in timestamp]
timestamp = df['2t_t0']
# una volta che hai buttato dentro quello che vuoi fai andare il codice (se non va fai andare anche la cella
# sopra che c'è un drop qui)
##################################################
# to save indexes for plotting later
timestamp_tensor = torch.tensor(timestamp, dtype = torch.float32)  # Encodes strings to integers

df.drop('timestamp', axis=1, inplace=True)

print("Input Dataframe Dim:", df.shape)


Input Dataframe Dim: (8320, 90)


In [186]:
# Custom Dataset
class TabularDataset(torch.utils.data.Dataset):
  def __init__(self, data_tensor, timestamp_tensor):
    self.data = data_tensor
    self.timestamp = timestamp_tensor

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    return self.data[index], 0, self.timestamp[index]                              # dummy label for compatibility

# Load and preprocess CSV data
#df = pd.read_csv("IT_NORD_VAE_input.csv")
#df.drop('timestamp', axis=1, inplace=True)
print("Input Dataframe Dim:", df.shape)

data_tensor = torch.tensor(df.values, dtype=torch.float32)

# Split the dataset into train, val, test
dataset = TabularDataset(data_tensor, timestamp_tensor)

total_size = len(dataset)
train_size = int(0.8 * total_size)                               # %80 training, %10 validation and %10 test
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

BATCH_SIZE = 128
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

Input Dataframe Dim: (8320, 90)


### Training 

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

INPUT_DIM = data_tensor.shape[1]
H_DIM = 64
NUM_EPOCHS = 50
BATCH_SIZE = 128
LR_RATE = 1e-3
Z_DIM_VALUES = [5]

all_histories = {}

for Z_DIM in Z_DIM_VALUES:
  print(f"\nTraining with Z_DIM = {Z_DIM}")
  model = VariationalAutoEncoder(INPUT_DIM, H_DIM, Z_DIM).to(DEVICE)
  optimizer = optim.Adam(model.parameters(), lr=LR_RATE)
  loss_fn = nn.MSELoss(reduction="sum")

  history = {
      'total_loss': [],
      'reconstruction_loss': [],
      'kl_div': [],
      'val_total_loss': [],
      'val_reconstruction_loss': [],
      'val_kl_div': [],
      'test_loss': None
  }
  outputs = []

  for epoch in range(NUM_EPOCHS):
    epoch_total = 0
    epoch_recon = 0
    epoch_kl = 0
    num_samples = 0

    model.train()
    loop = tqdm(train_loader, desc=f"Z_DIM = {Z_DIM} | Epoch {epoch+1}/{NUM_EPOCHS}")

    for i, (x_batch, _, _) in enumerate(loop):
      x_batch = x_batch.to(DEVICE)
      x_reconstructed, mu, sigma = model(x_batch)

      # Loss Function
      reconstruction_loss = loss_fn(x_reconstructed, x_batch)
      logvar = torch.log(sigma.pow(2) + 1e-8)
      kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
      loss = reconstruction_loss + kl_div

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loop.set_postfix(
          total_loss=loss.item() / x_batch.shape[0],
          reconstruction_loss=reconstruction_loss.item() / x_batch.shape[0],
          kl_div=kl_div.item() / x_batch.shape[0])

      epoch_total += loss.item()
      epoch_recon += reconstruction_loss.item()
      epoch_kl += kl_div.item()
      num_samples += x_batch.shape[0]

      if i == 0:
        outputs.append((epoch, x_batch.clone(), x_reconstructed.clone()))

    # Compute and store averages after each epoch
    history['total_loss'].append(epoch_total / num_samples)
    history['reconstruction_loss'].append(epoch_recon / num_samples)
    history['kl_div'].append(epoch_kl / num_samples)

    # Validation step
    model.eval()
    val_total = 0
    val_recon = 0
    val_kl = 0
    val_samples = 0

    with torch.no_grad():
      for x_val, _ , _ in val_loader:
        x_val = x_val.to(DEVICE)
        x_reconstructed, mu, sigma = model(x_val)
        loss_rec = loss_fn(x_reconstructed, x_val)
        logvar = torch.log(sigma.pow(2) + 1e-8)
        kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        loss = loss_rec + kl_div

        val_total += loss.item()
        val_recon += loss_rec.item()
        val_kl += kl_div.item()
        val_samples += x_val.size(0)

    history['val_total_loss'].append(val_total / val_samples)
    history['val_reconstruction_loss'].append(val_recon / val_samples)
    history['val_kl_div'].append(val_kl / val_samples)

    # Visualize latent space (t-SNE for high space visualization) (or PCA)
    
    if (epoch == (NUM_EPOCHS - 1)):
        model.eval()
        with torch.no_grad():
            zs = []
            ts = []
            for x_batch, _ , timestamp_batch in train_loader:
                x_batch = x_batch.to(DEVICE)
                mu, sigma = model.encode(x_batch)
                zs.append(mu.cpu().numpy())
                ts.append(timestamp_batch.numpy())
            zs = np.concatenate(zs, axis = 0)
            ts = np.concatenate(ts, axis = 0)

            # check for unique values
            unique_ts = np.unique(ts)
            ts_to_idx = {v: i for i, v in enumerate(unique_ts)}
            ts_idx = np.array([ts_to_idx[t] for t in ts])

            '''# Reduce to 2D
            if Z_DIM > 2:
                reducer = TSNE(n_components=2, perplexity=30, init='random', random_state=0)
                zs_2d = reducer.fit_transform(zs)
            else:
                zs_2d = zs

            plt.figure(figsize=(6, 5))
            scatter = plt.scatter(
                zs_2d[:, 0], zs_2d[:, 1],
                c=ts, cmap='tab20',  # Color by timestamp category
                alpha=0.6, s=5
            )
            plt.colorbar(scatter, label="Timestamp Category")
            plt.title(f"2D Projection of Latent Space at Epoch {epoch + 1} (Z_DIM={Z_DIM})")
            plt.xlabel("Component 1")
            plt.ylabel("Component 2")
            plt.grid(True)
            plt.tight_layout()
            plt.show()
            plt.close()'''

  # Final test set evaluation
  model.eval()
  test_loss = 0
  test_samples = 0
  with torch.no_grad():
    for x_test, _ , _ in test_loader:
      x_test = x_test.to(DEVICE)
      x_reconstructed, mu, sigma = model(x_test)
      loss_rec = loss_fn(x_reconstructed, x_test)
      logvar = torch.log(sigma.pow(2) + 1e-8)
      kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
      loss = loss_rec + kl_div
      test_loss += loss.item()
      test_samples += x_test.shape[0]

  history['test_loss'] = test_loss / test_samples
  print(f"Final Test Loss for Z_DIM = {Z_DIM}: {history['test_loss']:.4f}")

  all_histories[Z_DIM] = history


Training with Z_DIM = 5


Z_DIM = 5 | Epoch 1/50: 100%|█| 52/52 [00:00<00:00, 142.28it/s, kl_div=5.33, rec
Z_DIM = 5 | Epoch 2/50: 100%|█| 52/52 [00:00<00:00, 159.58it/s, kl_div=7.88, rec
Z_DIM = 5 | Epoch 3/50: 100%|█| 52/52 [00:00<00:00, 148.83it/s, kl_div=9.44, rec
Z_DIM = 5 | Epoch 4/50: 100%|█| 52/52 [00:00<00:00, 146.45it/s, kl_div=9.64, rec
Z_DIM = 5 | Epoch 5/50: 100%|█| 52/52 [00:00<00:00, 168.17it/s, kl_div=10.1, rec
Z_DIM = 5 | Epoch 6/50: 100%|█| 52/52 [00:00<00:00, 158.97it/s, kl_div=10.2, rec
Z_DIM = 5 | Epoch 7/50: 100%|█| 52/52 [00:00<00:00, 167.93it/s, kl_div=10.5, rec
Z_DIM = 5 | Epoch 8/50: 100%|█| 52/52 [00:00<00:00, 171.08it/s, kl_div=10.3, rec
Z_DIM = 5 | Epoch 9/50: 100%|█| 52/52 [00:00<00:00, 140.61it/s, kl_div=10.6, rec
Z_DIM = 5 | Epoch 10/50: 100%|█| 52/52 [00:00<00:00, 172.33it/s, kl_div=10.4, re
Z_DIM = 5 | Epoch 11/50: 100%|█| 52/52 [00:00<00:00, 188.06it/s, kl_div=10.6, re
Z_DIM = 5 | Epoch 12/50: 100%|█| 52/52 [00:00<00:00, 157.40it/s, kl_div=10.7, re
Z_DIM = 5 | Epoch 13/50: 100

# Attempt to use PCA for representing hidden space

In [ ]:
np.unique(ts)

In [ ]:
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
import itertools

X = zs
pca = PCA(n_components = 5)
zs_pca = pca.fit_transform(zs)
#print(zs_pca.explained_variance_ratio_)
print(np.shape(zs_pca))
plt.figure(figsize = (7.5, 5))


############################
# SOFIA
# qui ci sono un paio di cose che dovresti ritoccare per i plot
# colors handling
unique_ts = np.unique(ts)
# vengono usati solo i primi n colori (dove n numero di diverse categorie in timestamp
custom_colors = ['blue','green', 'orange','#006064','lightblue','#1B5E20', 'green', 'brown','orange','yellow','pink','red','#FFEB3B','#FAF4C4','#FAF4C4',]  
keep_colors = custom_colors[:len(unique_ts)]
custom_colormap = ListedColormap(custom_colors[:len(unique_ts)])
# legend
# qui dovresti scrivere a mano a che cosa corrisponde ogni categoria
# custom_labels = ['winter','spring','summer','autumn']
custom_labels = ['underproduction',
                 'balanced',
                 'overproduction']
#############################
#legend_handles = [
#    Patch(color=keep_colors[i], label=custom_labels[i]) for i in range(len(unique_ts))
#]

# Plot all 2D combinations of PCA components
combinations = list(itertools.combinations(range(zs_pca.shape[1]), 2))
for i, (x_idx, y_idx) in enumerate(combinations):
    plt.figure(figsize=(7.5, 5))
    
    scatter = plt.scatter(zs_pca[:, x_idx], zs_pca[:, y_idx], c=ts, cmap='jet', s=1.4)
    
    plt.title(f"PCA Latent Space (Z_DIM = {Z_DIM}) - PC{x_idx+1} vs PC{y_idx+1}")
    plt.xlabel(f"PC {x_idx+1}")
    plt.ylabel(f"PC {y_idx+1}")
    #plt.legend(handles=legend_handles, title="Classes", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.colorbar(label='Normalized solar progression')
    plt.tight_layout()
    plt.savefig(f"PCA_output/ZDIM_{Z_DIM}_PC{x_idx}_{y_idx}.png")
    plt.show()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharex=False)
ymin, ymax = 0, 140

# Plot of Total Loss
for z_dim, history in all_histories.items():
  axs[0].plot(history['total_loss'], label=f"Z_DIM = {z_dim}")
  axs[0].set_ylabel("Total Loss")
  axs[0].set_xlabel("Epoch")
  axs[0].set_title("Total Loss Across Epochs")
  axs[0].set_ylim(ymin, ymax)
  axs[0].legend()
  axs[0].grid(True)

# Plot of Reconstruction Loss
for z_dim, history in all_histories.items():
  axs[1].plot(history['reconstruction_loss'], label=f"Z_DIM = {z_dim}")
  axs[1].set_ylabel("Reconstruction Loss")
  axs[1].set_xlabel("Epoch")
  axs[1].set_title("Reconstruction Loss Across Epochs")
  axs[1].set_ylim(ymin, ymax)
  axs[1].legend()
  axs[1].grid(True)

# Plot of KL Divergence
for z_dim, history in all_histories.items():
  axs[2].plot(history['kl_div'], label=f"Z_DIM = {z_dim}")
  axs[2].set_xlabel("Epoch")
  axs[2].set_ylabel("KL Divergence")
  axs[2].set_title("KL Divergence Across Epochs")
  axs[2].set_ylim(ymin, ymax)
  axs[2].legend()
  axs[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
print(f"{'Z_DIM':<8} {'Total Loss':<13} {'Recon Loss':<13} {'KL-Divergence':<13}")
for z_dim, history in all_histories.items():
  print(f"{z_dim:<8} {history['total_loss'][-1]:<13.2f} {history['reconstruction_loss'][-1]:<13.2f} {history['kl_div'][-1]:<13.2f}")